In [1]:
import os, re
from dotenv import dotenv_values

from sqlalchemy import create_engine, text

from datetime import date,datetime,timedelta
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import requests

# Data Collection
Collection of the previously constituted data :
- Stations : info about the stations (a station is where the measure is done)
- Nitrates : all the measured nitrate concentrations
- Weather : weather history

In [2]:
# Settings from .env file

settings = dotenv_values()

# SQL CONFIG

settings = dotenv_values() # Loads settings from .env file
ROOT='..' # relative path to the root of the project

db_uri = (
    f"mysql+pymysql://{settings['SQL_USER']}:{settings['SQL_PWD']}"
    f"@{settings['SQL_HOST']}/{settings['SQL_DB']}"
    f"?ssl_ca={os.path.join(ROOT,settings['SQL_SSL_CA'])}"
    f"&ssl_cert={os.path.join(ROOT,settings['SQL_SSL_CERT'])}"
    f"&ssl_key={os.path.join(ROOT,settings['SQL_SSL_KEY'])}"
    f"&ssl_check_hostname=false"
)

engine = create_engine(db_uri,echo=False, future=False)

In [3]:
# Stations Dataframe

stations = pd.read_csv('../data/stations.csv')
stations.head()

,station_id,label,alt,river_id,river_label,mean_nitrate,lat,lon
0,6000990,BELRUPT,292,U---0000,La Saône,4.513699,48.090273,6.101941
1,6000998,MONTHUREUX-SUR-SAONE 2,241,U---0000,La Saône,6.428571,48.018251,5.941492
2,6000993,JONVELLE,230,U---0000,La Saône,9.433333,47.936380,5.923536
3,6001000,CENDRECOURT,213,U---0000,La Saône,9.233333,47.840261,5.917378
4,6003600,SCEY-SUR-SAONE-ET-ST-ALBI,210,U---0000,La Saône,8.421519,47.661731,5.972460


In [4]:
# weather dataframe

query = "SELECT * FROM weather ;"
weather = pd.read_sql_query(query,engine)
weather.day = pd.to_datetime(weather.day)
weather.head()

,day,station_id,temperature,precipitation,maxwind,description
0,2010-01-01,6000990,2.4,3.9,20.2,Heavy snow
1,2010-01-01,6000993,2.4,7.6,20.9,Heavy snow
2,2010-01-01,6000998,2.4,7.6,20.9,Heavy snow
3,2010-01-01,6001000,3.1,7.3,21.2,Heavy snow
4,2010-01-01,6002500,3.1,7.3,21.2,Heavy snow


In [5]:
# nitrates dataframe
nitrates = pd.read_csv('../data/nitrates_2011_2021.csv')
nitrates.day = pd.to_datetime(nitrates.day)
nitrates.head()

,station_id,day,measure
0,6000990,2011-01-19,4.5
1,6000990,2011-03-23,3.5
2,6000990,2011-05-17,4.7
3,6000990,2011-07-20,4.1
4,6000990,2011-09-21,4.5


# Data Constitution 
Constitution of the data that will be used by the model.  
  
Target : Nitrate concentration measured
Features :
- doy (day of the year, sinus and cosinus)
- weather the day of the measure (maxwind in kph, mean temperature in °C, precipitation in mm)
- weather the 60 days before the measure

In [6]:
DELTA = 60 # number of the previous days weather to include in the features

In [7]:
data = nitrates.copy()

In [8]:
# We add mean_nitrate to the df
mean_nitrate = stations[['station_id','mean_nitrate']]
data = pd.merge(data,mean_nitrate,how='inner',on='station_id')

In [9]:
# We add precipitations to the df
precipitations = weather[['station_id','day','precipitation']]
data = pd.merge(data,precipitations,how='inner',on=['station_id','day'])

# We add previous days precipitation
for delta in range(1,DELTA+1) :
    previous_prec = weather[['station_id','day','precipitation']].copy()
    previous_prec['day'] = previous_prec['day'] + timedelta(delta)
    previous_prec.columns = ['station_id','day',f'precipitation_{delta}']
    data = pd.merge(data,previous_prec,how='inner',on=['station_id','day'])

In [10]:
# We add temperature
temp = weather[['station_id','day','temperature']]
data = pd.merge(data,temp,how='inner',on=['station_id','day'])

# We add previous days temperature
for delta in range(1,DELTA+1) :
    previous_temp = weather[['station_id','day','temperature']].copy()
    previous_temp['day'] = previous_temp['day'] + timedelta(delta)
    previous_temp.columns = ['station_id','day',f'temperature_{delta}']
    data = pd.merge(data,previous_temp,how='inner',on=['station_id','day'])

In [11]:
# We add maxwind
maxwind = weather[['station_id','day','maxwind']]
data = pd.merge(data,maxwind,how='inner',on=['station_id','day'])

# We add previous days precipitation
for delta in range(1,DELTA+1) :
    previous_mw = weather[['station_id','day','maxwind']].copy()
    previous_mw['day'] = previous_mw['day'] + timedelta(delta)
    previous_mw.columns = ['station_id','day',f'maxwind_{delta}']
    data = pd.merge(data,previous_mw,how='inner',on=['station_id','day'])

data = data.copy() # dataframe defragmentation

In [12]:
# We add day_of_year
# Formats date
data['doy'] = data['day'].dt.dayofyear # doy = day of year

# Turns day of year to cyclical feature (sin,cos)
data['sin_doy'] = np.sin( (data['doy']-1) * 2 * np.pi / 365 )
data['cos_doy'] = np.cos( (data['doy']-1) * 2 * np.pi / 365 )

data = data.copy()

In [15]:
data.head()

,station_id,day,measure,mean_nitrate,precipitation,precipitation_1,precipitation_2,precipitation_3,precipitation_4,precipitation_5,...,maxwind_54,maxwind_55,maxwind_56,maxwind_57,maxwind_58,maxwind_59,maxwind_60,doy,sin_doy,cos_doy
0,6000990,2011-01-19,4.5,4.513699,1.1,1.3,0.1,0.0,1.7,0.8,...,20.2,23.8,15.5,14.4,10.1,12.6,12.6,19,0.304921,0.952378
1,6000990,2011-03-23,3.5,4.513699,0.0,0.0,0.0,0.0,14.4,1.2,...,20.2,19.1,16.9,24.5,9.7,15.8,18.7,82,0.984474,0.175531
2,6000990,2011-05-17,4.7,4.513699,3.6,1.6,7.3,5.1,0.0,3.3,...,11.9,14.8,12.2,19.8,14.8,19.8,16.2,137,0.717677,-0.696376
3,6000990,2011-07-20,4.1,4.513699,15.8,3.4,12.0,19.2,1.6,0.0,...,19.1,23.8,11.9,15.5,7.9,16.2,6.8,201,-0.296713,-0.954967
4,6000990,2011-09-21,4.5,4.513699,0.0,0.0,7.1,1.1,0.7,3.1,...,12.6,7.6,9.4,7.2,10.1,21.6,10.4,264,-0.982927,-0.183998


# Data export
- Export of the data as a pickle file, to be used to train the models

In [18]:
data.to_pickle('../data/prepared_data.pkl')

In [31]:
# asserts that the two dataframes are identical
test = pd.read_pickle('../data/prepared_data.pkl')
assert test.equals(data)